In [1]:
from application.inception import *
from application.WEIGHTS import *
from application.inception_resnet_v2 import *

weights = InceptionResNetV2_Weights()


In [2]:
img_input = keras.layers.Input(shape=[299, 299, 3])

In [3]:
# x = stem.conv0(img_input)
# print(x.shape)
# x = stem.conv1(x)
# print(x.shape)
# x = stem.conv2(x)
# print(x.shape)
# x = stem.max_pool3(x)
# print(x.shape)
# x = stem.conv4(x)
# print(x.shape)
# x = stem.conv5(x)
# print(x.shape)
# x = stem.max_pool6(x)
# print(x.shape)

In [3]:
model = InceptionResNetV2()
model.summary()

2022-02-02 19:32:15.371905: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-02 19:32:15.372038: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1 Pro

systemMemory: 32.00 GB
maxCacheSize: 10.67 GB

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 149, 149, 32) 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 149, 149, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 149, 149, 32) 0           batch_normalization[0][0]        
____________

In [6]:
keras_model = keras.applications.InceptionResNetV2(weights=None)
keras_model.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_650 (Conv2D)             (None, 149, 149, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_610 (BatchN (None, 149, 149, 32) 96          conv2d_650[0][0]                 
__________________________________________________________________________________________________
activation_649 (Activation)     (None, 149, 149, 32) 0           batch_normalization_610[0][0]    
________________________________________________________________________________

In [19]:
import kfood_dataset
batch_size = 32
paths = kfood_dataset.get_image_paths()

finding image paths...
shuffling...
paths ready!


In [20]:
train_set = kfood_dataset.make_kfood_dataset(paths[30000:], n_parse_threads=tf.data.AUTOTUNE, batch_size=batch_size, shuffle_buffer_size=len(paths[30000:]) // batch_size)
valid_set = kfood_dataset.make_kfood_dataset(paths[20000:30000], n_parse_threads=tf.data.AUTOTUNE, batch_size=batch_size, shuffle_buffer_size=10000 // batch_size)
test_set = kfood_dataset.make_kfood_dataset(paths[:20000], n_parse_threads=tf.data.AUTOTUNE, batch_size=None)

In [21]:
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True, decay=0.001)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [22]:
history = model.fit(train_set, steps_per_epoch=10,
          validation_data=valid_set, validation_steps=10, 
          epochs=1
          )

2022-02-02 19:18:08.612892: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.
2022-02-02 19:18:13.354933: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile


10/10 [==============================] - ETA: 0s - loss: 5.1736 - accuracy: 0.0063

2022-02-02 19:18:33.179323: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


10/10 [==============================] - 35s 2s/step - loss: 5.1736 - accuracy: 0.0063 - val_loss: 5.0203 - val_accuracy: 0.0094


In [23]:
model.save("model.h5")

NotImplementedError: Layer Stem has arguments in `__init__` and therefore must override `get_config`.